In [ ]:
import random

class GridWorld:
    def __init__(self, grid_size=4, terminal_states=None, gamma=0.9):
        """
        Simple grid world environment.
        :param grid_size: Size of the grid (grid_size x grid_size).
        :param terminal_states: List of terminal states as (row, col).
        :param gamma: Discount factor.
        """
        self.grid_size = grid_size
        self.terminal_states = terminal_states or [(grid_size - 1, grid_size - 1)]
        self.gamma = gamma
        self.reset()

    def reset(self):
        """Reset the environment to the initial state."""
        self.agent_pos = (0, 0)  # Start at top-left corner
        return self.agent_pos

    def step(self, action):
        """
        Take a step in the environment.
        :param action: Action to take (0=Up, 1=Right, 2=Down, 3=Left).
        :return: next_state, reward, done.
        """
        row, col = self.agent_pos
        moves = {0: (-1, 0), 1: (0, 1), 2: (1, 0), 3: (0, -1)}  # Up, Right, Down, Left
        dr, dc = moves[action]
        new_row = max(0, min(self.grid_size - 1, row + dr))
        new_col = max(0, min(self.grid_size - 1, col + dc))
        self.agent_pos = (new_row, new_col)

        if self.agent_pos in self.terminal_states:
            return self.agent_pos, 10, True  # Reward +10 for reaching terminal state
        return self.agent_pos, -1, False  # Step penalty -1

    def render(self):
        """Print the current state of the grid."""
        grid = [['.' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
        for r, c in self.terminal_states:
            grid[r][c] = 'T'  # Mark terminal states
        row, col = self.agent_pos
        grid[row][col] = 'A'  # Mark agent position
        print("\n".join([" ".join(row) for row in grid]))
        print()


class MonteCarloControl:
    def __init__(self, env, gamma=0.9, epsilon=0.1):
        """
        Monte Carlo Control algorithm implementation.
        :param env: The environment object.
        :param gamma: Discount factor.
        :param epsilon: Exploration rate.
        """
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = {}  # Action-value function Q(s, a)
        self.returns = {}  # Returns for state-action pairs
        self.policy = {}  # Policy π(s)

        for row in range(env.grid_size):
            for col in range(env.grid_size):
                state = (row, col)
                self.Q[state] = [0, 0, 0, 0]  # Initialize Q-values for all actions
                self.returns[state] = {a: [] for a in range(4)}  # Initialize returns
                self.policy[state] = [0.25, 0.25, 0.25, 0.25]  # Epsilon-greedy policy

    def epsilon_greedy(self, state):
        """Select an action using epsilon-greedy policy."""
        if random.random() < self.epsilon:
            return random.choice(range(4))  # Explore: random action
        return self.Q[state].index(max(self.Q[state]))  # Exploit: best action

    def generate_episode(self):
        """Generate an episode using the current policy."""
        episode = []
        state = self.env.reset()
        done = False
        while not done:
            action = self.epsilon_greedy(state)
            next_state, reward, done = self.env.step(action)
            episode.append((state, action, reward))
            state = next_state
        return episode

    def optimize_policy(self, num_episodes=1000):
        """Run Monte Carlo Control to optimize the policy."""
        for _ in range(num_episodes):
            episode = self.generate_episode()

            # Compute returns for the episode
            G = 0  # Initialize return
            visited = set()  # Track visited state-action pairs in this episode
            for state, action, reward in reversed(episode):
                G = reward + self.gamma * G
                if (state, action) not in visited:
                    visited.add((state, action))
                    self.returns[state][action].append(G)
                    self.Q[state][action] = sum(self.returns[state][action]) / len(self.returns[state][action])

            # Update the policy
            for state in self.policy:
                best_action = self.Q[state].index(max(self.Q[state]))
                for action in range(4):
                    if action == best_action:
                        self.policy[state][action] = 1 - self.epsilon + (self.epsilon / 4)
                    else:
                        self.policy[state][action] = self.epsilon / 4
        return self.policy

env = GridWorld(grid_size=4, terminal_states=[(3, 3)])
mc_control = MonteCarloControl(env)

policy = mc_control.optimize_policy(num_episodes=1000)

print("Optimal Policy (State -> Best Action):")
for state in sorted(mc_control.policy):
    best_action = mc_control.Q[state].index(max(mc_control.Q[state]))
    print(f"State {state}: Action {best_action}")

env.render()

Optimal Policy (State -> Best Action):
State (0, 0): Action 1
State (0, 1): Action 1
State (0, 2): Action 1
State (0, 3): Action 2
State (1, 0): Action 0
State (1, 1): Action 0
State (1, 2): Action 0
State (1, 3): Action 2
State (2, 0): Action 1
State (2, 1): Action 2
State (2, 2): Action 3
State (2, 3): Action 2
State (3, 0): Action 1
State (3, 1): Action 1
State (3, 2): Action 1
State (3, 3): Action 0
. . . .
. . . .
. . . .
. . . A

